#  Sokoban Informado
## Avaliação Contínua 2
### Introdução à Inteligência Artificial, edição 2024/25, Dep. Informática FCUL
### Entrega: 22 de Outubro (1m antes da meia-noite)

***

## Introdução e Objetivos

Nesta avaliação pretende-se atingir dois objetivos, ambos relacionados com a procura informada:

**Objetivo Heuristica:** Dada a implementação em Python da modelização do Sokoban como um problema de procura num grafo, devem completá-la implementando uma nova função heurística (descrita em baixo) que estime o menor custo até ao objectivo, para que seja usada por algoritmos de procura informada.

**Objetivo Algoritmos:** Deve implementar dois novos algoritmos de procura informada (descritos em baixo), o Beam Search e uma variante a que chamamos Iterative Widening Beam Search, que funcionem em qualquer problema modelado num espaço de estados.

Vão precisar dos módulos seguintes (distribuídos juntamente com o enunciado):
* `searchPlus.py`
* `utils.py`
* `sokoban.py`
* `ProblemaGrafoHs.py`

***

## O puzzle Sokoban

Considere o problema do Sokoban que foi tema da Avaliação Contínua 1 e que relembramos aqui. 

O puzzle Sokoban é jogado tipicamente numa grelha discreta de células quadradas, onde cada célula é uma parede ou chão navegável. Algumas das células de chão contêm caixas e outras estão marcadas como lugares de armazenamento das caixas.

O jogador (Sokoban) está confinado ao tabuleiro e pode movimentar-se ortogonalmente para células adjacentes, que sejam navegáveis e vazias - o Sokoban não tem super poderes, não podendo atravessar nem paredes nem caixas. Algumas células vazias não são consideradas navegáveis porque, mesmo não sendo lugares de armazenamento, causam deadlocks muito simples caso uma caixa seja empurrada para lá (porque já não será possível voltar a mover essa caixa, e assim o jogo nunca terminará). 

O Sokoban pode também empurrar caixas que estejam ao seu lado e pode fazê-lo para células vazias adjacentes. As caixas não podem ser empurradas se ao seu lado, na orientação do movimento, estiver uma parede ou outra caixa. 

O número de caixas é sempre igual ao número de lugares de armazenamento. O puzzle fica resolvido quando todos os lugares de armazenamento forem ocupados por caixas.


## Modelização do Sokoban em Python

A seguir apresentamos a implementação da classe `Sokoban` para a modelização do problema como um grafo de estados, que utilizámos como referência para a realização dos testes da Avaliação Contínua 1.

Incluímos já duas heurísticas `h_inutil_1` e `h_inutil_2` que, apesar de poderem ser usadas para testar os algoritmos de procura informada, não são lá muito úteis. A heurística `h_inutil_1` limita-se a reconhecer que um estado é final e a calcular a distância de Manhattan entre o Sokoban e a caixa mais longínqua (retorna 0 na presença de um estado final e a referida distância em todos os outros casos). A heurística `h_inutil_2` limita-se a contar o número de caixas que ainda não estão arrumadas em lugares de armazenamento.


### Representação do estado, informação estática do problema e objetivos
Representamos as células da grelha em pares de coordenadas (linha,coluna) onde a primeira linha é a linha 0 e a primeira coluna é também 0.

Vamos representar um estado como um dicionário. A tabela tem os seguintes pares chaves e valores:
* "sokoban": as coordendas da célula onde está o Sokoban
* "caixas": o conjunto com as coordenadas das células onde estão situadas as caixas.

Na verdade, o estado é subclasse do tipo dicionário do Python porque queremos usar a versão em grafo da procura e para isso precisamos de guardar os estados em conjuntos e para isso eles terão de ser hashable.


Como informação estática do problema teremos tudo o que não se altera com as acções:
* O mapa da grelha que indica o nº de colunas para cada linha, que é importante para o display em modo txt e linhas e respectivas colunas.
* O conjunto de células navegáveis

que são guardados em dois atributos da instância do problema:

* o mapa da grelha (um dicionário quen indica para cada linha quantas colunas existem)
* as células navegáveis

Na construção extraímos do txt a informação do estado e geramos o dicionário para o guardar no seu atributo  `initial`.

Os objectivos (lugares de armazenamento das caixas) também estão inscritos no input em txt do construtor e são guardados para serem usados pela função `goal_test`. São guardados no atributo `goal` da instância do problema.

* "navegáveis": as células navegáveis do puzzle.
* "objectivos": o conjunto das células que terão de ter as caixas para resolver o problema.
* "sokoban": as coordendas da célula onde está o Sokoban.
* "caixas": o conjunto com as coordenadas das células onde estão situadas as caixas.
* "mapa": a tabela com os índices das linhas e para cada uma delas o nº de colunas.

É a função `conv_txt` que converte a representação do puzzle de texto num dicionário, que é usado para distribuir a informação revelada no texto pelo estado (a posição inicial do Sokoban e as posições das caixas), pela informação estática do problema (o mapa de linhas e colunas, as paredes, as casas navegáveis) e pelo objectivo (as posições para onde queremos que o Sokoban transporte as caixas). 
Porque precisamos das paredes? Na verdade porque temos células '#' e células que não são navegáveis, com o símbolo ' '. É só para podermos redesenhar o puzzle de input que aparece com esses dois símbolos.
 

### As classes `Sokoban` e `EstadoSokoban` e funções auxiliares

In [ ]:
from searchPlus import *
import copy


def manhattan(p,q):
    (x1,y1) = p
    (x2,y2) = q
    return abs(x1-x2) + abs(y1-y2)


def conv_txt(txt):
    linhas=txt.split('\n')
    linhas=[linha for linha in linhas] 
    dados={'caixas':set(), 'objectivos':set(), 'navegáveis':set(), 'paredes':set()}
    map_puzzle={}
    y=0
    for linha in linhas[:-1]:
        x=0
        for c in linha:
            if c=='$':
                dados['caixas'].add((y,x))
                dados['navegáveis'].add((y,x))
            elif c=='@':
                dados['sokoban']=(y,x)
                dados['navegáveis'].add((y,x))
            elif c=='+':
                dados['sokoban']=(y,x)
                dados['objectivos'].add((y,x))
                dados['navegáveis'].add((y,x))
            elif c=='*':
                dados['caixas'].add((y,x))
                dados['objectivos'].add((y,x))
                dados['navegáveis'].add((y,x))
            elif c=='.':
                dados['navegáveis'].add((y,x))
            elif c=='o':
                dados['navegáveis'].add((y,x))
                dados['objectivos'].add((y,x))
            elif c=='#':
                dados['paredes'].add((y,x))
            x+=1
        map_puzzle[y]=x
        y+=1
    dados["mapa"]=map_puzzle
    return dados


class EstadoSokoban(dict): 
    def __hash__(self):
        #print(self['caixas'])
        ll=list(self['caixas'])
        ll.append(self['sokoban'])
        ll.sort()
        return hash(str(ll))
    
    def __lt__(self,other):
        """Um estado é sempre menor do que qualquer outro, para desempate na fila de prioridades"""
        return True


linha1= "  #####\n"
linha2= "###...#\n"
linha3= "#o@$..#\n"
linha4= "###.$o#\n"
linha5= "#o##..#\n"
linha6= "#.#...##\n"
linha7= "#$.....#\n"
linha8= "#......#\n"
linha9= "########\n"
mundoStandard=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9


class Sokoban(Problem):
    """O """
    
    dict_actions = {'N': (-1,0), 'W': (0,-1), 'E': (0,1), 'S': (1,0)}

    def __init__(self, initial=None,goal=None,situacaoInicial=mundoStandard):
        """A partir do puzzle em txt geramos um dicionários de onde extráimos:
        * o estado, as posições objectivo, as casas bavegáveis, o mapa e as paredes.
        No estado ficamos com a célula (linha,coluna) com a posição do Sokoban e 
        o conjunto das posições das caixas."""
        dados=conv_txt(situacaoInicial)
        self.initial=EstadoSokoban()
        self.initial['sokoban']=dados['sokoban']
        self.initial['caixas']=dados['caixas']
        self.goal=dados['objectivos']
        self.navegaveis=dados['navegáveis']
        self.mapa=dados['mapa']
        self.paredes=dados['paredes']
        self.proibidas=self.calc_traps()
        
    def possible(self,sokoban,caixas,deltas):
        l,c=sokoban
        #print('sokoban:',(l,c))
        dl,dc=deltas
        #print('deltas:',(dl,dc))
        l1,c1=l+dl,c+dc
        #print('next:',(l1,c1),'In caixas?',(l1,c1) in caixas,'In navegáveis?',(l1,c1) in self.navegaveis)
        if not (l1,c1) in caixas:
            #print('sai na primeira')
            return (l1,c1) in self.navegaveis
        else:
            #print('sai no else')
            l2,c2=l1+dl,c1+dc
            #print('nextnext:',(l2,c2),'In caixas?',(l2,c2) in caixas,'In navegáveis?',(l2,c2) in self.navegaveis)
            return (l2,c2) in self.navegaveis and (l2,c2) not in self.proibidas and (l2,c2) not in caixas
   
        
    def its_a_trap(self,cel):
        viz=[]
        num_viz=0
        l,c=cel
        deltas=[self.dict_actions[a] for a in self.dict_actions]
        for (dl,dc) in deltas:
            if (l+dl,c+dc) in self.navegaveis:
                num_viz+=1
                viz.append((l+dl,c+dc))
                #print(cel,viz,(l+dl,c+dc))
            if num_viz > 2:
                #print(False)
                return False
        if num_viz == 1:
            return True
        l1,c1=viz[0]
        l2,c2=viz[1]
        return l1!=l2 and c1!=c2
        
    def calc_traps(self):
        the_traps=set()
        for n in self.navegaveis:
            if not n in self.goal and self.its_a_trap(n):
                the_traps.add(n)
        return the_traps
        
    def actions(self, state):
        sokoban=state['sokoban']
        return [action for action in self.dict_actions if self.possible(sokoban,state['caixas'],self.dict_actions[action])]
    
    def result(self, state, action):
        clone=copy.deepcopy(state)
        l,c=clone['sokoban']
        dl,dc=self.dict_actions[action]
        l1,c1=l+dl,c+dc
        clone['sokoban']=(l1,c1)
        if (l1,c1) in clone['caixas']:
            l2,c2=l1+dl,c1+dc
            clone['caixas'].remove((l1,c1))
            clone['caixas'].add((l2,c2))
        return clone

    def goal_test(self,state):
        for caixa in state['caixas']:
            if caixa not in self.goal:
                return False
        return True

    def executa(self,state,actions):
        """Partindo de state, executa a sequência (lista) de acções (em actions) e devolve o último estado"""
        nstate=state
        for a in actions:
            nstate=self.result(nstate,a)
        return nstate

    def simbolo(self,estado,cel):
        (l,c)=cel
        if estado['sokoban']==(l,c):
            if (l,c) in self.goal:
                return '+'
            return "@"
        elif (l,c) in estado['caixas']:
            if (l,c) in self.goal:
                return '*'
            return '$'
        elif (l,c) in self.goal:
            return 'o'
        elif (l,c) in self.navegaveis:
            return '.'
        elif (l,c) in self.paredes:
            return '#'
        return ' '
        
    def display(self, state):
        """Devolve a grelha em modo txt"""
        puzzle=''
        for l in self.mapa:
            for c in range(self.mapa[l]):
                puzzle+=self.simbolo(state,(l,c))
            puzzle +='\n'
        return puzzle

    def h_inutil_1(self, node):
        """Heurística inútil pois só reconhece estados finais e, caso contrário,
        devolve a distância de Manhattan entre o Sokoban e a caixa mais distante."""
        clone=copy.deepcopy(node.state)
        ## Satisfaz objectivo?
        if self.goal_test(clone):
            return 0
        max_distancia_sokoban = 0
        sokoban = clone['sokoban']
        for caixa in clone['caixas']:
            d_sokoban = manhattan(sokoban,caixa)  # ignoramos possíveis obstáculos
            if d_sokoban > max_distancia_sokoban:
                max_distancia_sokoban = d_sokoban
        return max_distancia_sokoban
    
    def h_inutil_2(self, node):
        """Heurística inútil pois só conta quantas caixas falta arrumar nos lugares de armazenamento."""
        clone=copy.deepcopy(node.state)
        ## Satisfaz objectivo?
        if self.goal_test(clone):
            return 0
        n_caixas_desarrumadas = len(clone['caixas'])
        for caixa in clone['caixas']:
            if caixa in self.goal:
                n_caixas_desarrumadas -= 1
        return n_caixas_desarrumadas
    
    def h_util(self, node):
        """Para cada objetivo (lugar de armazenamento), calcula a distância de Manhattan à caixa mais próxima
        que ainda não foi alocada, ignorando a existência de paredes e/ou obstáculos, e aloca essa caixa ao objetivo.
        O valor da heurística é a soma todas estas distâncias + a distância entre o sokoban e a caixa mais longínqua
        que ainda não está arrumada. Se estamos num estado final, devolve 0."""
        pass


## Objetivo Heurística

Reparem que `h_util` está por preencher. É essa a função que devem implementar. **Não mexam** em mais nada da classe, ou poderão obter um comportamento que não é o esperado, nomeadamente quando há empates nos valores da heurística.

Esta heurística deve associar cada objetivo (pela ordem em que ficam guardados em `self.goal`) à caixa mais próxima que ainda não foi associada, considerando a distância de Manhattan entre a caixa e o objetivo. Em caso de empate entre duas ou mais caixas, deve-se associar a primeira caixa (também pela ordem em que estão guardadas).

Depois de somar todas as distâncias entre os objetivos e as respetivas caixas, deve ainda adicionar a distância entre o Sokoban e a caixa mais longínqua que ainda não se encontra arrumada num lugar de armazenamento.

Nota: A associação entre objetivos e caixas não garante que a soma das distâncias é a mínima possível. Não devem tentar minimizar esta soma alterando a ordem pela qual associam os objetivos.

## Objetivo Algoritmos

Devem implementar o algoritmo de procura informada **Beam Search**. Este algoritmo não garante uma solução ótima. Na verdade, nem garante uma solução. É um algoritmo que limita o número de nós expandidos em cada nível de profundidade, como se iluminasse os caminhos possíveis com um foco de largura limitada (*beam width*) e deixasse todos os outros às escuras. Eis como funciona:

* Em cada nível de profundidade, expande apenas os W melhores nós (W é o *beam width*, ou a largura do foco de luz) de acordo com uma função *f*. Todos os outros nós são cortados, e esses caminhos nunca serão explorados.
* Ao expandir um nó: se encontrar um caminho melhor do que outro que estava entre os W a serem expandidos (portanto, na profundidade acima), este outro é cortado e já não será expandido; se encontrar um caminho melhor do que outro já existente na mesma profundidade, substitui-o tal como acontece nos outros algoritmos de procura em grafo.
* Nota: em cada profundidade, a expansão dos nós deve ser feita por ordem crescente da função *f*. Os casos de empate são automaticamente resolvidos pelo código fornecido, não precisam de se preocupar com isso (nem devem, ou poderão obter resultados diferentes do esperado).

Na figura seguinte, que representa uma procura Beam Search com W = 2, apenas os nós pretos foram expandidos.

<img src="./Imagens/ii1.png" alt="Drawing" style="width: 400px;"/>


Devem também implementar uma variante do Beam Search a que chamamos **Iterative Widening Beam Search**. Este algoritmo aplica a procura Beam Search da seguinte forma: Começa com W = 1 e vai aumentando W iterativamente até que seja encontrada uma solução (e podem assumir que há sempre solução). A implementação deve devolver a solução encontrada, o W em que se encontrou a solução, e o número acumulado de nós explorados no total das iterações.

Para implementar estes dois algoritmos, completem o que está por preencher nas funções `beam_search_plus_count` e `IW_beam_search` no código abaixo. Para a função `beam_search_plus_count` podem basear-se no código do `best_first_graph_search_plus_count` disponível em `searchPlus.py` (procura em grafo) e adaptá-lo.

**Não mexam** na função `beam_search`, que se limita a invocar a `beam_search_plus_count` com f = custo + heurística. Mas podem (e devem) usá-la na implementação da `IW_beam_search`.

In [ ]:

def beam_search_plus_count(problem, W, f):
    """Beam Search: search the nodes with the best W scores in each depth.
       Return the solution and how many nodes were expanded."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    pass


def beam_search(problem, W, h=None):
    """Beam graph search with f(n) = g(n)+h(n).
    You need to specify W and the h function when you call beam_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return beam_search_plus_count(problem, W, lambda n: n.path_cost + h(n))


def IW_beam_search(problem, h):
    """IW_beam_search (Iterative Widening Beam Search) começa com beam width W=1 e aumenta W iterativamente até
    se obter uma solução. Devolve a solução, o W com que se encontrou a solução, e o número total (acumulado desde W=1)
    de nós expandidos. Assume-se que existe uma solução."""
    pass


### A classe `ProblemaGrafoHs`

Obviamente que estes novos algoritmos de procura devem funcionar em qualquer problema modelado num espaço de estados.

Vamos agora recordar a classe `ProblemaGrafoHs` dos guiões das aulas, onde temos um grafo com custos associados a cada arco e valores heurísticos para cada nó, definidos no método `h1`. Para este enunciado foi acrescentado o método `h2` que define uma outra heurística (ver no código abaixo). Consideramos I como estado inicial e F como estado final. 


In [ ]:
class ProblemaGrafoHs(Problem) :
    grafo = {'I':{'A':2,'B':6},
             'A':{'C':2,'D':4,'I':2},
             'B':{'D':1,'F':5,'I':6},
             'C':{},
             'D':{'C':2,'F':2},
             'F':{}}

    def __init__(self,initial = 'I', final = 'F') :
        super().__init__(initial,final)
        
    def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        """
        return accao.split()[-1]
    
    def path_cost(self, c, state1, action, state2):
        return c + self.grafo[state1][state2]

    def h1(self,no) : 
        """Uma heurística é uma função de um estado.
        Nesta implementação, é uma função do estado associado ao nó
        (objecto da classe Node) fornecido como argumento.
        """
        h = {'I':7, 'A':6,'B':2,'C':4,'D':2,'F':0}
        return h[no.state]
    
    def h2(self,no) : 
        h = {'I':10, 'A':4,'B':5,'C':5,'D':20,'F':0}
        return h[no.state]

***

## Submissão

### Quizz
Cada grupo deve completar as implementações pedidas e testá-las no link do *quizz* **Avaliação Contínua 2** que está na página da disciplina, introduzindo aí o vosso código. 

Esse *quizz* é constituído por duas perguntas. As implementações da heurística `h_util` e dos algoritmos de procura `beam_search_plus_count` e `IW_beam_search` são avaliadas através de um conjunto de testes automáticos visíveis e mais alguns testes escondidos, valendo no total 1.75 valores (0.5 para a heurística, 1.25 para os algoritmos). Em ambos os casos, os testes visíveis valem 30% da nota e os invisíveis valem 70%.

Podem ir verificando o código (botão check) e submeterem as vezes que quiserem (por ambos os elementos do grupo), sendo a submissão com melhor nota a que será considerada. Qualquer tentativa não manualmente submetida é automaticamente submetida no fecho do prazo.

### Prazo
A submissão fecha às 23:59 de 3a feira, 22 de Outubro

### Ficheiro Python
Simultaneamente é necessario submeter o ficheiro Python que contém todo o código, na página da disciplina. Só queremos uma submissão por grupo. Esse ficheiro deve chamar-se *SokobanInformado_IIA_24_25_grupoXX.py* em que substituem XX pelo número do grupo. 